## Web-Scraping of the Wikipedia page using BeautifulSoup

In [1]:
# Importing necessary libraries
from bs4 import BeautifulSoup
import requests
from concurrent.futures import ThreadPoolExecutor
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, REAL, TEXT, INTEGER
from sqlalchemy.orm import sessionmaker
import json
import re

BASE_URL = 'https://en.wikipedia.org' # Base URL of Wikipedia
HEADERS = {'User-Agent': 'Mozilla/5.0'} # Headers to simulate a browser visit

# Sending GET request to the Wikipedia page containing the list of highest-grossing films
response = requests.get(BASE_URL + '/wiki/List_of_highest-grossing_films', headers=HEADERS)
soup = BeautifulSoup(response.content, "lxml") # BeautifulSoup object for parsing the data
data = [] # Empty list to hold the films data

# Finding the necessary table with the data that's needed and creating variables to hold some data-related values
table = soup.findAll('table', {'class': 'wikitable plainrowheaders'})[1]
prev_year = 0
film_links = []

# Iterating through necessary rows in the table
for row in table.findAll("tr")[1:-1]:
    # Extracting Film Title, Box Office, Budget
    columns = row.find_all('td')[:3]
    # Trimming and storing text values from the selected columns
    cols = [ele.text.strip() for ele in columns]

    # Handling of specific problems related to some titles/years
    if cols[0] == "Harry Potter and the Philosopher's Stone" or cols[0] == "Snow White and the Seven Dwarfs":
        cols[1] = cols[1].split("(")[0].strip()[:-2]

    if cols[0] == "Harry Potter and the Deathly Hallows \u2013 Part 2":
        cols[2] = cols[2][:-1]

    if row.find('th') is not None:
        year = row.find('th').text.strip()
        cols.append(year)
        prev_year = year
    else:
        cols.append(prev_year)

    # Finding the link to the film's details and storing it
    new_href = columns[0].find('a', href=True)
    if new_href:
        film_links.append(new_href['href'])
    
    # Appending the gathered columns data to the data list
    data.append(cols)

# Function to scrape film details from the individual film page
def scrape_film_details(link):
    film_data = [] # List to store the film's details
    # New request (to parse the film's personal page)
    response2 = requests.get(BASE_URL + link, headers=HEADERS)
    soup2 = BeautifulSoup(response2.content, 'lxml')
    # Finding the table with necessary data
    table_film_related = soup2.find('table', {'class': 'infobox vevent'})
    
    # Extracting the director's name
    if table_film_related:
        director = table_film_related.find_all('tr')[2].find('td').text.strip()
        ind = 0
        # Handling of specific problems related to some director values
        if (len(director) == 0) or ("Directed" not in table_film_related.find_all('tr')[2].find('th').text.strip()):
            for part in table_film_related.find_all('tr'):
                th_element = part.find('th')
                if th_element is not None:
                    th_text = th_element.text.strip()
                    if "Directed" in th_text:
                        break
                ind += 1
            director = table_film_related.find_all('tr')[ind].find('td').text.strip() 

        # Appending the director's name to the film data
        film_data.append(director)

        # Extracting the country
        country_row_index = len(table_film_related.find_all('tr')) - 4
        country = table_film_related.find_all('tr')[country_row_index].find('td').text.strip()

        # Handling of specific problems related to some country values
        if len(country.split()) >= 2 and country.split()[1] == 'minutes':
            country_row_index = len(table_film_related.find_all('tr')) - 3
            country = table_film_related.find_all('tr')[country_row_index].find('td').text.strip()
        
        # Appending the country to the film data
        film_data.append(country)

    return film_data

# Using ThreadPoolExecutor to make concurrent requests (to make the process of parsing faster)
with ThreadPoolExecutor(max_workers=5) as executor:
    film_details = list(executor.map(scrape_film_details, film_links))

# Combination of the collected details (director's name, country) with the original data (title, box office, budget)
for index, details in enumerate(film_details):
    if index < len(data):
        data[index].extend(details)

# Function to clean a string by removing specified parts
def clean_string(original_string, parts_to_remove):
    cleaned_string = original_string
    for part in parts_to_remove:
        cleaned_string = cleaned_string.replace(part, '')
    cleaned_string = cleaned_string.strip()
    return cleaned_string

# Function to format the director names appropriately
def director_format_fix(str, last_names, words_to_remove):
    final = []
    str = clean_string(str, words_to_remove)
    f = False
    if str == "John FordMervyn LeRoyJoshua Logan":
        return "John Ford\nMervyn LeRoy\nJoshua Logan"
    for part in str.split('\n'):
        f = False
        for name in last_names:
            if name in part:
                f = True
                break
        if not f:
            final.append(re.sub(r'(?<=[a-z.])(?=[A-Z])', '\n', re.sub(r"\[.*?\]", '', part)))
        else:
            final.append(part)
    return '\n'.join(final)

# Words to remove from director descriptions
words_to_remove = ['British & French:', 'American:', 'German:', '(action sequences)',
                   '(uncredited)', 'Live action:', 'Animation:', 'Dialogue directors:', '[1]',
                   'Supervising Directors', 'Sequence Directors',
                   'Uncredited:', 'Supervising Director', 'Sequence Director']
# Words to keep unchanged in director descriptions
last_names = ['McCarey', 'DeMille', 'LeRoy', 'McTiernan']
structured_data = [] # List to hold structured data for each film

# Iterating over the collected data to clean, structure it, and store in the structured_data list
for item in data:
    if len(item) >= 6:
        film_record = {
            "title": item[0],
            "box_office": float(re.sub(r'[^0-9.]', '', re.split(r'[–/(\s]', item[1])[0])),
            "budget": float(re.sub(r'[^0-9.]', '', re.split(r'[–/(\s]', item[2])[0])),
            "release_year": int(item[3]),
            "director": director_format_fix(item[4], last_names, words_to_remove),
            "country": re.sub(r'(?<=[a-z])(?=[A-Z])', '\n', re.sub(r"\[.*?\]", '', item[5]))
        }
        structured_data.append(film_record)

/var/folders/m0/9dr3jfm12bj1fkg2xjqh62jc0000gn/T/ipykernel_8103/691294321.py:21: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  table = soup.findAll('table', {'class': 'wikitable plainrowheaders'})[1]
/var/folders/m0/9dr3jfm12bj1fkg2xjqh62jc0000gn/T/ipykernel_8103/691294321.py:26: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  for row in table.findAll("tr")[1:-1]:


## Creation of the database using SQLAlchemy

In [2]:
# Setting up SQLAlchemy to create a SQLite database
engine = create_engine('sqlite:///highest-grossing_films.db', echo=True)
Base = declarative_base()

# Defining Film class mapping to the film table
class Film(Base):
    __tablename__ = 'films_data'
    id = Column(INTEGER, primary_key=True, autoincrement=True)
    title = Column(TEXT)
    release_year = Column(INTEGER)
    director = Column(TEXT)
    box_office = Column(REAL)
    budget = Column(REAL)
    country = Column(TEXT)

    # Method to represent the Film object in a readable format
    def __repr__(self):
        return "<Film(title='%s', director='%s', year='%s')>" % (self.title, self.director, self.release_year)

# Creating the table in the database
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Delete any existing records from the table (to avoid duplicates)
session.query(Film).delete()

# Adding each structured film record to the session
for w in structured_data:
    session.add(Film(**w))

# Committing the changes to the database
session.commit()

2025-02-25 00:38:06,759 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-25 00:38:06,760 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("films_data")
2025-02-25 00:38:06,760 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-25 00:38:06,761 INFO sqlalchemy.engine.Engine COMMIT
2025-02-25 00:38:06,763 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-25 00:38:06,764 INFO sqlalchemy.engine.Engine DELETE FROM films_data
2025-02-25 00:38:06,764 INFO sqlalchemy.engine.Engine [generated in 0.00033s] ()
2025-02-25 00:38:06,770 INFO sqlalchemy.engine.Engine INSERT INTO films_data (title, release_year, director, box_office, budget, country) VALUES (?, ?, ?, ?, ?, ?)
2025-02-25 00:38:06,770 INFO sqlalchemy.engine.Engine [generated in 0.00036s] ('The Birth of a Nation', 1915, 'D. W. Griffith', 50000000.0, 110000.0, 'United States')
2025-02-25 00:38:06,771 INFO sqlalchemy.engine.Engine INSERT INTO films_data (title, release_year, director, box_office, budget, country) VALUES (?, ?

/var/folders/m0/9dr3jfm12bj1fkg2xjqh62jc0000gn/T/ipykernel_8103/670517983.py:3: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


## Export of the database content to a JSON file for further visualization

In [ ]:
# Querying all film records from the database
films = session.query(Film).all()

# Creating a list of dictionaries to store film data
films_data = [{
    'id': film.id,
    'title': film.title,
    'release_year': film.release_year,
    'director': film.director,
    'box_office': float(film.box_office),
    'budget': float(film.budget),
    'country': film.country
} for film in films]

# Writing the films data to a JSON file
with open('highest_grossing_films.json', 'w') as json_file:
    json.dump(films_data, json_file, indent=4)

# Closing the session
session.close()

2025-02-25 00:38:10,271 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-25 00:38:10,275 INFO sqlalchemy.engine.Engine SELECT films_data.id AS films_data_id, films_data.title AS films_data_title, films_data.release_year AS films_data_release_year, films_data.director AS films_data_director, films_data.box_office AS films_data_box_office, films_data.budget AS films_data_budget, films_data.country AS films_data_country 
FROM films_data
2025-02-25 00:38:10,275 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ()
2025-02-25 00:38:10,284 INFO sqlalchemy.engine.Engine ROLLBACK
